In [2]:
cd ~/MultiFidelity-ProcessOpt/1.\ Code


/users/k23070952/MultiFidelity-ProcessOpt/1. Code


/users/k23070952/CODE/0. SSH files/jvenv/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/users/k23070952/CODE/0. SSH files/jvenv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from ShortCutDesign import ShortCutDesign 
from RigorousDesign import RigorousDesign 
import os
import numpy as np
import pandas as pd
import numpy as np
import random

import itertools
import multiprocessing
import time
from tqdm import tqdm

In [4]:
# Define the bounds as per the given function
bounds = [
    (1, 50),           # Number of stages in extractor
    (1e-4, 0.9999), (1e-4, 0.9999),  # Light key, heavy key for extract distiller
    (1e-4, 0.9999), (1e-4, 0.9999),  # Light key, heavy key for acetic_acid_purification
    (273, 350),         # Temperature for HX
    (1e-4, 0.9999), (1e-4, 0.9999)   # Light key, heavy key for raffinate distiller
]

# 각 파라미터를 10개의 그리드로 나누기
grid_points = [np.linspace(low, high, 10) for low, high in bounds]

# 가능한 모든 조합 생성 (그리드 서치)
param_combinations = list(itertools.product(*grid_points))

print(f"Total search space: {len(param_combinations)} combinations")

Total search space: 100000000 combinations


In [5]:
# Define the bounds as per the given function
bounds = [
    (10, 50),           # Number of stages in extractor
    (0.95, 0.9999), (0.95, 0.9999),  # Light key, heavy key for extract distiller
    (0.95, 0.9999), (0.95, 0.9999),  # Light key, heavy key for acetic_acid_purification
    (300, 350),         # Temperature for HX
    (0.95, 0.9999), (0.95, 0.9999)   # Light key, heavy key for raffinate distiller
]

# 각 파라미터를 10개의 그리드로 나누기
grid_points = [np.linspace(low, high, 10) for low, high in bounds]

# 가능한 모든 조합 생성 (그리드 서치)
param_combinations = list(itertools.product(*grid_points))

print(f"Total search space: {len(param_combinations)} combinations")

Total search space: 100000000 combinations


In [14]:

# Define the bounds
bounds = [
    (1, 50),           # Number of stages in extractor
    (1e-4, 0.9999), (1e-4, 0.9999),  # Light key, heavy key for extract distiller
    (1e-4, 0.9999), (1e-4, 0.9999),  # Light key, heavy key for acetic_acid_purification
    (273, 350),         # Temperature for HX
    (1e-4, 0.9999), (1e-4, 0.9999)   # Light key, heavy key for raffinate distiller
]

# Number of random samples you want to generate
n_samples = 1000000

# Generate random samples
param_combinations = [
    [random.uniform(low, high) for (low, high) in bounds]
    for _ in range(n_samples)
]

print(f"Total random search space: {len(param_combinations)} combinations")


Total random search space: 1000000 combinations


In [6]:

# ✅ 파일 경로 설정
results_file = '../3. Data/results_250327.csv'
error_file = '../3. Data/errors_250327.csv'

# ✅ Shortcut 및 Rigorous 결과 계산 함수 
def calculate_shortcut(_init_params):
    shortcut_model = ShortCutDesign()
    shortcut_results = shortcut_model.shortcut_results(_init_params)
    return shortcut_results

def calculate_rigorous(_init_params):
    shortcut_results = calculate_shortcut(_init_params)
    if shortcut_results['CAPEX'] == 0:
        return (shortcut_results, [0, 0, 0])

    if shortcut_results['CAPEX'] is None or np.isnan(shortcut_results['CAPEX']):
        return (shortcut_results, [0, 0, 0])

    RigorousCal = RigorousDesign()
    Rigorous_results = RigorousCal.check_results(_init_params, shortcut_results)
    return (shortcut_results, Rigorous_results)

# ✅ 평가 함수 (병렬 실행용)
def evaluate(params):
    try:
        # 계산 수행
        shortcut_obj, rigorous_obj = calculate_rigorous(params)

        # 결과 데이터 정리
        record = list(params) + [
            shortcut_obj['CAPEX'], shortcut_obj['OPEX'], shortcut_obj['AceticAcidWt'], shortcut_obj['SplitRatio'], 
            shortcut_obj['boilup_1'], shortcut_obj['N_stages_1'], shortcut_obj['feed_stage_1'], 
            shortcut_obj['boilup_2'],shortcut_obj['N_stages_2'],shortcut_obj['feed_stage_2'],
            shortcut_obj['boilup_3'],shortcut_obj['N_stages_3'],shortcut_obj['feed_stage_3'],shortcut_obj['shortcut_time'],
            rigorous_obj[0], rigorous_obj[1], rigorous_obj[2], rigorous_obj[3]
        ]

        # 실시간 CSV 저장
        df = pd.DataFrame([record])
        df.to_csv(results_file, mode='a', header=not os.path.exists(results_file), index=False)

        return record

    except Exception as e:
        error_df = pd.DataFrame([[params, str(e)]], columns=['Params', 'Error'])
        error_df.to_csv(error_file, mode='a', header=not os.path.exists(error_file), index=False)
        return None


def evaluate_short_only(params):
    try:
        # 계산 수행
        shortcut_obj = calculate_shortcut(params)

        # 결과 데이터 정리
        record = list(params) + [
            shortcut_obj['CAPEX'], shortcut_obj['OPEX'], shortcut_obj['AceticAcidWt'], shortcut_obj['SplitRatio'], 
            shortcut_obj['boilup_1'], shortcut_obj['N_stages_1'], shortcut_obj['feed_stage_1'], 
            shortcut_obj['boilup_2'],shortcut_obj['N_stages_2'],shortcut_obj['feed_stage_2'],
            shortcut_obj['boilup_3'],shortcut_obj['N_stages_3'],shortcut_obj['feed_stage_3'],shortcut_obj['shortcut_time'],
            0,0,0,0
        ]

        # 실시간 CSV 저장
        df = pd.DataFrame([record])
        df.to_csv(results_file, mode='a', header=not os.path.exists(results_file), index=False)

        return record

    except Exception as e:
        error_df = pd.DataFrame([[params, str(e)]], columns=['Params', 'Error'])
        error_df.to_csv(error_file, mode='a', header=not os.path.exists(error_file), index=False)
        return None


In [7]:
len(param_combinations)

100000000

In [8]:
# ✅ 병렬 실행
if __name__ == "__main__":
    # SLURM이 할당한 CPU 개수 가져오기 (없으면 기본값 4)
    num_cpus = int(os.getenv("SLURM_CPUS_ON_NODE", "4"))
    print(f"Using {num_cpus} CPUs for parallel processing.")

    # ✅ 실행 시작 시간 기록
    start_time = time.time()
    completed_tasks = 0

    # ✅ 멀티프로세싱 풀 생성 및 실행
    results = []
    with multiprocessing.Pool(processes=num_cpus) as pool:
        for result in pool.imap_unordered(evaluate_short_only, param_combinations[:]):  # 50만 개 실행
            if result:
                results.append(result)
                completed_tasks += 1

            # ✅ 100개 완료될 때마다 진행 상태 출력
            if completed_tasks % 100 == 0:
                elapsed = time.time() - start_time
                print(f"Completed {completed_tasks} tasks in {elapsed:.2f} seconds")

    # ✅ 실행 종료 시간 기록
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"\nGrid search completed in {elapsed_time:.2f} seconds.")

Using 64 CPUs for parallel processing.
 ##### An instance of the 'BlackBox' class  has been initialised!
(10.0, 0.95, 0.95, 0.95, 0.95, 300.0, 0.9666333333333333, 0.9721777777777778)
 ##### An instance of the 'BlackBox' class  has been initialised! ##### An instance of the 'BlackBox' class  has been initialised!

 ##### An instance of the 'BlackBox' class  has been initialised!(10.0, 0.95, 0.95, 0.95, 0.95, 300.0, 0.9666333333333333, 0.9666333333333333) ##### An instance of the 'BlackBox' class  has been initialised!(10.0, 0.95, 0.95, 0.95, 0.95, 300.0, 0.9555444444444444, 0.9777222222222222)


 ##### An instance of the 'BlackBox' class  has been initialised!(10.0, 0.95, 0.95, 0.95, 0.95, 300.0, 0.9721777777777778, 0.9555444444444444)
 ##### An instance of the 'BlackBox' class  has been initialised!

 ##### An instance of the 'BlackBox' class  has been initialised!(10.0, 0.95, 0.95, 0.95, 0.95, 300.0, 0.9555444444444444, 0.9555444444444444) ##### An instance of the 'BlackBox' class  ha